<a href="https://colab.research.google.com/github/ahnnoah/study/blob/main/Colab/2022/4_%EC%98%81%ED%99%94%ED%8F%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN으로 영화평 감성 분석하기
안노아

In [ ]:
# 1번 셀

# 패키지 수입
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from time import time

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import confusion_matrix, f1_score


In [ ]:
# 2번 셀

# 하이퍼 파라미터 설정
MY_WORDS = 5000          # 사전 안에 단어 수 (사용 빈도수로 sorting)
MY_LEN = 200             # 영화평 통일된 길이
MY_EMBED = 32            # 임베딩 차원
MY_HIDDEN = 64           # LSTM의 차원

MY_EPOCH = 10            # 반복 학습 수
MY_BATCH = 200           # 한번에 처리하는 데이터 수

In [ ]:
# 3번 셀

# 데이터 불러오기
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=MY_WORDS)

# 샘플 출력
# IMDB의 영화평은 token 처리 되어 있음
# 감성: 0은 부정, 1은 긍정
print('학습용 데이터 0번 영화평:', X_train[0]) 
print('학습용 데이터 0번 감성:', Y_train[0]) 

# 데이터 모양 확인
print('학습용 입력 데이터:', X_train.shape)
print('학습용 출력 데이터:', Y_train.shape)
print('평가용 입력 데이터:', X_test.shape)
print('평가용 출력 데이터:', Y_test.shape)


학습용 데이터 0번 영화평: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
학습용 데이터 0번 감성: 1
학습용 입력 데이터: (25000,)
학습용 출력 데이터: (25000,)


In [ ]:
# 4번 셀

# 단어를 정수로 전환하기
word_to_id = imdb.get_word_index()
print('the의 token:', word_to_id['the'])
print('virus의 token:', word_to_id['virus'])

# 정수를 단어로 전환하기
id_to_word = {}
for word, id in word_to_id.items():
    id_to_word[id] = word

print('token 1의 단어:', id_to_word[1])
print('token 2의 단어:', id_to_word[2])
print('token 3310의 단어:', id_to_word[3310])


the의 token: 1
virus의 token: 3310
token 1의 단어: the
token 2의 단어: and
token 3310의 단어: virus


In [ ]:
# 5번 셀

# 영화평 단어로 전환
# 영화평에 특수 문자 3개 추가되어 있음
# 0: 패딩
# 1: 영화평 시작
# 2: 삭제 단어
def decode(review):
    output = []

    for i in review:
        word = id_to_word.get(i - 3, '???')
        output.append(word)
    print(output)

decode(X_train[333])


['???', 'with', 'title', 'like', 'this', 'you', 'know', 'you', 'get', 'pretty', 'much', 'lot', 'of', 'junk', 'acting', 'bad', 'script', 'bad', 'director', 'bad', '???', 'bad', 'br', 'br', 'movie', 'make', 'lot', 'of', 'noise', 'that', 'really', 'not', 'music', 'and', 'lot', 'of', 'people', '???', 'movie', 'make', 'bad', 'racial', 'stereotype', 'why', 'come', 'every', 'movie', 'with', 'black', 'hero', 'have', 'drug', '???', 'why', 'come', 'hero', 'always', 'have', 'to', 'dance', 'to', 'be', 'success', 'why', 'come', 'famous', '???', 'always', 'have', 'to', 'be', 'in', 'dance', 'movie', 'why', 'come', 'letter', 's', "can't", 'be', 'in', 'title', 'br', 'br', 'hollywood', 'need', 'to', 'stop', 'dumb', 'down', 'audience', 'and', 'make', 'movie', 'that', 'have', 'people', 'with', 'brain', 'who', 'know', 'how', 'speak', 'proper', 'english', 'br', 'br', 'do', 'self', 'favor', 'and', 'not', 'go', 'see']


In [ ]:
# 6번 셀

# 영화평 길이 통일
for i in range(10):
    print('영화평:', i, '길이:', len(X_train[i]))

X_train = pad_sequences(X_train,
                        truncating='pre',
                        padding='pre',
                        maxlen=MY_LEN)

for i in range(10):
    print('영화평:', i, '길이:', len(X_train[i]))

X_test = pad_sequences(X_test,
                        truncating='pre',
                        padding='pre',
                        maxlen=MY_LEN)    


영화평: 0 길이: 218
영화평: 1 길이: 189
영화평: 2 길이: 141
영화평: 3 길이: 550
영화평: 4 길이: 147
영화평: 5 길이: 43
영화평: 6 길이: 123
영화평: 7 길이: 562
영화평: 8 길이: 233
영화평: 9 길이: 130
영화평: 0 길이: 200
영화평: 1 길이: 200
영화평: 2 길이: 200
영화평: 3 길이: 200
영화평: 4 길이: 200
영화평: 5 길이: 200
영화평: 6 길이: 200
영화평: 7 길이: 200
영화평: 8 길이: 200
영화평: 9 길이: 200


In [ ]:
# 7번 셀

# 데이터 모양 확인
print('학습용 입력 데이터:', X_train.shape)
print('학습용 출력 데이터:', Y_train.shape)
print('평가용 입력 데이터:', X_test.shape)
print('평가용 출력 데이터:', Y_test.shape)


학습용 입력 데이터: (25000, 200)
학습용 출력 데이터: (25000,)
평가용 입력 데이터: (25000, 200)
평가용 출력 데이터: (25000,)


In [ ]:
# 8번 셀

# RNN 구현
model = Sequential()

# 임베딩 층 추가
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED,
                    input_length=MY_LEN))

# LSTM 추가
model.add(LSTM(units=MY_HIDDEN))

# 출력층
model.add(Dense(units=1,
                activation='sigmoid'))

# RNN 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 184,897
Trainable params: 184,897
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 9번 셀

# RNN 학습
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

print('학습 시작')
begin = time()

model.fit(X_train,
          Y_train,
          epochs=MY_EPOCH,
          batch_size=MY_BATCH,
          verbose=1)

end = time()
print('총 학습 시간:', end - begin)

학습 시작
Epoch 1/10
125/125 [==============================] - 4s 14ms/step - loss: 0.4959 - acc: 0.7420
Epoch 2/10
125/125 [==============================] - 2s 14ms/step - loss: 0.2726 - acc: 0.8904
Epoch 3/10
125/125 [==============================] - 2s 14ms/step - loss: 0.2278 - acc: 0.9114
Epoch 4/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1967 - acc: 0.9258
Epoch 5/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1758 - acc: 0.9327
Epoch 6/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1602 - acc: 0.9408
Epoch 7/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1391 - acc: 0.9482
Epoch 8/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1270 - acc: 0.9538
Epoch 9/10
125/125 [==============================] - 2s 14ms/step - loss: 0.1093 - acc: 0.9611
Epoch 10/10
125/125 [==============================] - 2s 14ms/step - loss: 0.0987 - acc: 0.9654
총 학습 시간: 19.467148780822754


In [ ]:
# 10번 셀

# RNN 평가하기
score = model.evaluate(X_test,
                       Y_test)

print('최종 정확도:', score[1])


782/782 [==============================] - 4s 5ms/step - loss: 0.4605 - acc: 0.8590
최종 정확도: 0.859000027179718


In [ ]:
# 11번 셀

# RNN으로 예측
pred = model.predict(X_test)

# 평가용 데이터 샘플 예측
print('영화평:', decode(X_test[0]))
print('정답:', Y_test[0])
print('예측:', pred[0])


['???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', '???', 'please', 'give', 'this', 'one', 'a', 'miss', 'br', 'br', '???', '??

In [ ]:
# 12번 셀

# 확률 결과를 이진수로 전환
print(pred)
pred = (pred > 0.5)
print(pred)

# 혼동 행렬
print(confusion_matrix(Y_test,
                       pred))

# F1 점수
print(f1_score(Y_test,
               pred,
               average='micro'))


[[False]
 [ True]
 [ True]
 ...
 [False]
 [False]
 [False]]
[[False]
 [ True]
 [ True]
 ...
 [False]
 [False]
 [False]]
[[10794  1706]
 [ 1819 10681]]
0.859
